In [2]:
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import time

from utils import JacobiGL, constructV, constructVx, constructD, z2x, z2x_dx, x2z, x2z_dz, JacobiP


# Boundary value problems

## a)

In [3]:
# eps_vec = [0.1, 0.01, 0.001]
eps = 0.1
x0, xN = 0, 1
g0, gN = 0, 0

def uexact(x, eps):
    return (np.exp(-x/eps) + (x-1) - np.exp(-1/eps)*x) / (np.exp(-1/eps) - 1)

### Legendre Tau Method 

In [ ]:
N = 20
alpha, beta = 0, 0  # Legendre

# Find Gauss Lobatto nodes
x = JacobiGL(alpha, beta, N)


##### Find fhat values #####
# Quadrature weights
w = 2 / (N * (N+1)) / np.power(JacobiP(x, alpha, beta, N), 2)

f_samples = np.ones_like(x)

# Precompute Jacobi polynomials for all k=0..N-1
# Shape: (N, len(x))
Phi = np.vstack([JacobiP(x, alpha, beta, k) for k in range(N)])

# Compute gamma_k for all k at once
gamma = np.sum(Phi**2 * w, axis=1)

# Compute numerator = <u, phi_k> for all k
numerators = np.sum((f_samples * Phi) * w, axis=1)

fhat = numerators / gamma

##### Calculate ghat #####
nvec = np.arange(2, N-2)
ghat = np.zeros_like(fhat, dtype = float)
ghat[2:N-2] = fhat[3:N-1] / (eps * (2*nvec + 3)) - fhat[1:N-3] / (eps * (2*nvec - 1))
ghat[0:2] = fhat[0:2]
# Since ghat already 0, I don't do anything for boundary conditions

##### Set up system matrix A, such that A uhat = ghat #####
A = np.zeros((N,N), dtype=float)

# Add diagonal
A[2:N-2, 2:N-2] = np.diag(-1/(eps*(2*nvec-1)*(2*nvec+3))) 

# Add sub and super diagonals
A += np.diag(np.concatenate((1/(eps*(2*nvec-1)**2), [0, 0])), k=-2) 
A += np.diag(np.concatenate(([0], 1/(eps*(2*nvec-1)), [0, 0])), k=-1)
A -= np.diag(np.concatenate(([0, 0], 1/(eps*(2*nvec+3)), [0])), k=1)
A += np.diag(np.concatenate(([0, 0], 1/(eps*(2*nvec+3)**2))), k=2) 

# Add two first rows using truncated versions of the sum-formulations
nvec = np.arange(N)

A[0, 2::2] = -eps/2 * (nvec[2::2] * (nvec[2::2] + 1))
A[0, 1::2] = - np.ones_like(nvec[1::2], dtype=float)
A[1, 3::2] = - eps * 3/2 * (nvec[3::2] * (nvec[3::2] + 1) - 2)
A[1, 2::2] = - 3 * np.ones_like(nvec[2::2], dtype=float)

# Add last two rows using boundary conditions
A[-2:] = np.vstack([JacobiP([-1,1], alpha, beta, n) for n in range(N)]).transpose()